In [1]:
import pandas as pd
from openai import OpenAI
import pickle
import time
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('dodiom_tr_test.csv')

In [3]:
df

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,adım atmak,"Sen bu adama adım mı attın ki, o sana yakınlaş...",idiom,0.75,3,0,1,"[3, 5]","['adım', 'attın']","['se', 'bu', 'ada', 'adım', 'mı', 'atmak', 'ki...","['Sen', 'bu', 'adama', 'adım', 'mı', 'attın', ..."
1,adım atmak,Bütün zorluklara rağmen yeni bir işe adım attı.,idiom,1.00,3,0,0,"[6, 7]","['adım', 'attı']","['bütün', 'zor', 'rağmen', 'yen', 'bir', 'işem...","['Bütün', 'zorluklara', 'rağmen', 'yeni', 'bir..."
2,adım atmak,Adım attığı bu hayatın başına ne gibi olaylar ...,idiom,0.75,3,0,1,"[0, 1]","['Adım', 'attığı']","['adım', 'atmak', 'bu', 'hayat', 'baş', 'ne', ...","['Adım', 'attığı', 'bu', 'hayatın', 'başına', ..."
3,adım atmak,"Bölüm başkanı, akademik dünyaya adım atan genç...",idiom,0.50,1,0,1,"[5, 6]","['adım', 'atan']","['bölüm', 'başka', ',', 'akademik', 'Dünya', '...","['Bölüm', 'başkanı', ',', 'akademik', 'dünyaya..."
4,adım atmak,Yeni bir deneyime adım atıyorum.,idiom,0.75,3,0,1,"[3, 4]","['adım', 'atıyorum']","['yen', 'bir', 'deneyim', 'adım', 'atmak', '.']","['Yeni', 'bir', 'deneyime', 'adım', 'atıyorum'..."
...,...,...,...,...,...,...,...,...,...,...,...
2079,üzerinde durmak,Bu ipin üzerinde duramıyorum.,nonidiom,1.00,5,0,0,"[2, 3]","['üzerinde', 'duramıyorum']","['bu', 'ip', 'Üzer', 'durmak', '.']","['Bu', 'ipin', 'üzerinde', 'duramıyorum', '.']"
2080,üzerinde durmak,İpin üzerinde tek ayağıyla duruyordu.,nonidiom,1.00,5,0,0,"[1, 4]","['üzerinde', 'duruyordu']","['ip', 'Üzer', 'tek', 'Ayağ', 'durmak', '.']","['İpin', 'üzerinde', 'tek', 'ayağıyla', 'duruy..."
2081,üzerinde durmak,Bilgisayarın üzerinde kitapları duruyor,nonidiom,1.00,4,0,0,"[1, 3]","['üzerinde', 'duruyor']","['bilgisayar', 'Üzer', 'kitap', 'durmak']","['Bilgisayarın', 'üzerinde', 'kitapları', 'dur..."
2082,üzerinde durmak,Üzerinde çilekler duran tartı bana uzattı.,nonidiom,1.00,1,0,0,"[0, 2]","['Üzerinde', 'duran']","['Üzer', 'Çilek', 'durmak', 'tartı', 'ban', 'u...","['Üzerinde', 'çilekler', 'duran', 'tartı', 'ba..."


In [3]:
idioms = df["submission"].tolist()

In [4]:
client = OpenAI(api_key="OpenAI KEY")

In [5]:
idioms

['Sen bu adama adım mı attın ki, o sana yakınlaşabilsin.',
 'Bütün zorluklara rağmen yeni bir işe adım attı.',
 'Adım attığı bu hayatın başına ne gibi olaylar getireceğinden habersizdi.',
 'Bölüm başkanı, akademik dünyaya adım atan genç araştırmacılarla tek tek ilgilendi.',
 'Yeni bir deneyime adım atıyorum.',
 'Petrol yasasında geri adım mı atılacak.',
 'Atmadan önce adımını iki kere düşün.',
 'Büyükçe bir adım atarak çukurun üstünden atladı.',
 'Ayaklarımq kara sular indi, bir adım daha atacak halim yok inan.',
 '5 adım attıktan sonra sağa dön ve 20 adım daha at.',
 'Asansöre adım atarken kabini kontrol ediyorum',
 'Imzamı adımın yanına attım.',
 'Bebeğimiz bugün ilk adımını attı 🤗',
 'Adım atacak yer yoktu.',
 'Beraberliğin çok önemli olduğunun altını çizdi.',
 'Altı çizerek söylüyorum artık ne kimsenin yanında ne de arkasındayım.',
 'Altını çizmek istediğim bir konu var',
 'Bir şeyin altını çizmek zor geliyorsa üstünü de karalamamak gerekir.',
 'Altını çizerek söylüyorum.',
 'Basın

In [6]:
results = []
for t in tqdm(range(len(idioms))):  # Loop over the range of idioms
    i = idioms[t]  # Access each idiom directly using the loop index
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
    {"role": "system", "content": 
        "Senin görevin Türkçe konuşan bir dil modelisin. Cevap verirken sadece verilen yönergeye göre işlem yap ve başka bir şey yazma."
    },
    {"role": "user", "content": 
        "Aşağıda verilen cümlede bir deyim olup olmadığını tespit etmeni istiyorum. Eğer cümlede deyim varsa, yalnızca '1' yaz. Eğer deyim yoksa yalnızca '0' yaz. Yalnızca 0 ya da 1 yanıtı ver, başka bir şey yazma. Cümle:\n" + i
    }
]
    )
    results.append(completion)  # Append the result to the list

  0%|          | 0/2084 [00:00<?, ?it/s]

In [7]:
results

[ChatCompletion(id='chatcmpl-AFkydeAXBNGpS8guXnohy9j7HMNtH', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1728318863, model='gpt-4-turbo-2024-04-09', object='chat.completion', service_tier=None, system_fingerprint='fp_82bed303cf', usage=CompletionUsage(completion_tokens=1, prompt_tokens=179, total_tokens=180, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))),
 ChatCompletion(id='chatcmpl-AFkydNFIRZqoXsBEHd59SP5I0NPpC', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='0', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1728318863, model='gpt-4-turbo-2024-04-09', object='chat.completion', service_tier=None, system_fingerprint='fp_68a5bb159e', usage=C

In [8]:
with open('gpt-4-turbo_1.pkl', 'wb') as f:
      pickle.dump(results, f)

In [4]:
labels = df["category"].tolist()

In [10]:
df = pd.DataFrame([i.choices[0].message.content for i in results])
df.to_csv('results_gpt-4-turbo.csv', index=False)

In [10]:
df

,0
0,1
1,0
2,0
3,0
4,0
...,...
2079,0
2080,0
2081,0
2082,0


In [9]:
for i in [i for i, x in enumerate(df[0].tolist()) if x == "1\n"]:
    df[0][i] = "1"

KeyError: 0

In [13]:
df

,0
0,1
1,0
2,0
3,0
4,0
...,...
2079,0
2080,0
2081,0
2082,0


In [14]:
gpt_labels = [int(i) for i in df.iloc[:, 0].tolist()]


In [11]:
human_labels = [1 if i == "idiom" else 0 for i in labels]

In [16]:
gpt_labels

[1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,


In [17]:
human_labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,


In [18]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(human_labels, gpt_labels, digits=4)

print(report)


              precision    recall  f1-score   support

           0       0.66      0.79      0.72      1115
           1       0.69      0.54      0.61       969

    accuracy                           0.67      2084
   macro avg       0.68      0.67      0.66      2084
weighted avg       0.68      0.67      0.67      2084



In [17]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(human_labels, gpt_labels, digits=4)

print(report)


              precision    recall  f1-score   support

           0     0.6634    0.7937    0.7227      1115
           1     0.6933    0.5366    0.6050       969

    accuracy                         0.6742      2084
   macro avg     0.6784    0.6652    0.6639      2084
weighted avg     0.6773    0.6742    0.6680      2084

